In [57]:
from woocommerce import API
import os
import pandas as pd
from dotenv import load_dotenv
import time
from rich.progress import Progress, TextColumn, BarColumn, TimeElapsedColumn, TimeRemainingColumn

In [13]:
# Get the current working directory (where the notebook is running)
current_dir = Path(os.getcwd())

# Navigate two levels up to find .env
dotenv_path = current_dir.parents[0] / ".env"

load_dotenv(dotenv_path)

# Access variables
CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")

In [29]:
wcapi = API(
    url="http://shoplens.local",
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    wp_api=True,
    version="wc/v3",
)

In [35]:
df = pd.read_csv("./sampled_fashion_categories.csv")
df.shape

(2100, 10)

In [36]:
df.head(5)

,asin,title,imgUrl,productURL,stars,reviews,price,isBestSeller,boughtInLastMonth,categoryName
0,B0CH6GQLQY,Trainers Black Girls Children's Shoes Dance Sh...,https://m.media-amazon.com/images/I/41V+H6No6E...,https://www.amazon.co.uk/dp/B0CH6GQLQY,0.0,0,0.65,False,0,Ballet & Dancing Footwear
1,B0CKR3TRZZ,Latin Dance Shoes Women's Stiletto Heel Practi...,https://m.media-amazon.com/images/I/61aYXPunjN...,https://www.amazon.co.uk/dp/B0CKR3TRZZ,0.0,0,7.76,False,0,Ballet & Dancing Footwear
2,B0030Z9MC4,"Capezio Jazz Shoes, Versatile Black Jazz Shoes...",https://m.media-amazon.com/images/I/71zUAMnOH6...,https://www.amazon.co.uk/dp/B0030Z9MC4,4.1,21,13.00,False,0,Ballet & Dancing Footwear
3,B0C2T6JXCT,s.lemon Women Black Hard Soled Dance Shoes Lea...,https://m.media-amazon.com/images/I/714Ppf42ZS...,https://www.amazon.co.uk/dp/B0C2T6JXCT,4.5,2,33.89,False,0,Ballet & Dancing Footwear
4,B0CHRVR8GG,2024 New -Y9K Fashion Latin Dancing Shoes Wome...,https://m.media-amazon.com/images/I/61NQ-oaR4W...,https://www.amazon.co.uk/dp/B0CHRVR8GG,0.0,0,6.21,False,0,Ballet & Dancing Footwear


In [40]:
df_categories = pd.DataFrame({'id': None, 'categoryName': df['categoryName'].unique()})
df_categories

,id,categoryName
0,None,Ballet & Dancing Footwear
1,None,Basketball Footwear
2,None,Beauty
3,None,Boating Footwear
4,None,Boxing Shoes
5,None,Climbing Footwear
6,None,Cricket Shoes
7,None,Cycling Shoes
8,None,Equestrian Sports Boots
9,None,Golf Shoes


In [48]:
# Create a progress bar with custom columns
with Progress(
    TextColumn("[cyan]Creating Categories..."),
    BarColumn(),
    TextColumn("{task.completed}/{task.total}"),
    TimeElapsedColumn(),
    TimeRemainingColumn(),
) as progress:

    task = progress.add_task("Creating Categories...", total=len(df_categories))

    for index, row in df_categories.iterrows():
        data = {"name": row["categoryName"]}

        # Send API request
        response = wcapi.post("products/categories", data).json()

        # Extract category ID if available
        category_id = response.get("id")
        if category_id:
            df_categories.at[index, "id"] = category_id  # Update ID in the DataFrame

        progress.update(task, advance=1)  # Update progress bar
        time.sleep(1)  # Add a delay to avoid rate limiting

# Save the updated DataFrame with IDs
df_categories.to_csv("categories.csv", index=False)

print("✅ Category creation process completed!")


Output()

✅ Category creation process completed!


In [49]:
df_categories

,id,categoryName
0,20,Ballet & Dancing Footwear
1,21,Basketball Footwear
2,22,Beauty
3,23,Boating Footwear
4,24,Boxing Shoes
5,25,Climbing Footwear
6,26,Cricket Shoes
7,27,Cycling Shoes
8,28,Equestrian Sports Boots
9,29,Golf Shoes


In [52]:
# Merge products with categories based on categoryName
df = df.merge(df_categories[['categoryName', 'id']], on='categoryName', how='left')

# Rename the 'id' column to 'categoryId'
df.rename(columns={'id': 'categoryId'}, inplace=True)

df.head(5)

,asin,title,imgUrl,productURL,stars,reviews,price,isBestSeller,boughtInLastMonth,categoryName,categoryId
0,B0CH6GQLQY,Trainers Black Girls Children's Shoes Dance Sh...,https://m.media-amazon.com/images/I/41V+H6No6E...,https://www.amazon.co.uk/dp/B0CH6GQLQY,0.0,0,0.65,False,0,Ballet & Dancing Footwear,20
1,B0CKR3TRZZ,Latin Dance Shoes Women's Stiletto Heel Practi...,https://m.media-amazon.com/images/I/61aYXPunjN...,https://www.amazon.co.uk/dp/B0CKR3TRZZ,0.0,0,7.76,False,0,Ballet & Dancing Footwear,20
2,B0030Z9MC4,"Capezio Jazz Shoes, Versatile Black Jazz Shoes...",https://m.media-amazon.com/images/I/71zUAMnOH6...,https://www.amazon.co.uk/dp/B0030Z9MC4,4.1,21,13.00,False,0,Ballet & Dancing Footwear,20
3,B0C2T6JXCT,s.lemon Women Black Hard Soled Dance Shoes Lea...,https://m.media-amazon.com/images/I/714Ppf42ZS...,https://www.amazon.co.uk/dp/B0C2T6JXCT,4.5,2,33.89,False,0,Ballet & Dancing Footwear,20
4,B0CHRVR8GG,2024 New -Y9K Fashion Latin Dancing Shoes Wome...,https://m.media-amazon.com/images/I/61NQ-oaR4W...,https://www.amazon.co.uk/dp/B0CHRVR8GG,0.0,0,6.21,False,0,Ballet & Dancing Footwear,20


In [77]:
df.to_csv("./sampled_fashion_categories.csv", index=False)

In [81]:
products_df = pd.read_csv("sampled_fashion_categories.csv")

# Add a column for product IDs
products_df.insert(0, "id", None)

with Progress(
    TextColumn("[cyan]Creating Products..."),
    BarColumn(),
    TextColumn("{task.completed}/{task.total}"),
    TimeElapsedColumn(),
    TimeRemainingColumn(),
) as progress:
    
    task = progress.add_task("Creating Products...", total=len(products_df))
    
    for index, row in products_df.iterrows():
        description_text = f"{row['title']}. {row['title']}"
        data = {
            "name": row["title"],
            "type": "simple",
            "regular_price": str(row["price"]*180),
            "description": description_text,
            "short_description": row["title"],
            "categories": [{"id": row["categoryId"]}],
            "images": [{"src": row["imgUrl"]}],
            "sku": row["asin"],
            "featured": bool(row["isBestSeller"]),
            "meta_data": [
                {"key": "stars", "value": str(row["stars"])},
                {"key": "reviews", "value": str(row["reviews"])},
                {"key": "boughtInLastMonth", "value": str(row["boughtInLastMonth"])}
            ]
        }
        
        try:
            # Send API request
            response = wcapi.post("products", data).json()
        
            # Extract product ID if available
            product_id = response.get("id")
            if product_id:
                products_df.at[index, "id"] = product_id  # Update ID in DataFrame
            else:
                print(f"⚠️ No ID returned for ASIN {row['asin']}")
        
        except Exception as e:
            print(f"⚠️ Request failed for ASIN: {row['asin']} - {e}")
            continue  # Skip to the next iteration


        progress.update(task, advance=1)  # Update progress bar
        time.sleep(0.25)  # Reduce sleep to 250ms

# Save the updated DataFrame with product IDs
products_df.to_csv("products_with_ids.csv", index=False)

print("✅ Product creation process completed!")


Output()

⚠️ Request failed for ASIN: B00V636FU4 - HTTPConnectionPool(host='shoplens.local', port=80): Read timed out. (read 
timeout=5)

⚠️ Request failed for ASIN: B0CJ386B4H - HTTPConnectionPool(host='shoplens.local', port=80): Read timed out. (read 
timeout=5)

⚠️ Request failed for ASIN: B00R4KTO9M - HTTPConnectionPool(host='shoplens.local', port=80): Read timed out. (read 
timeout=5)

⚠️ Request failed for ASIN: B0CHWFPYXZ - HTTPConnectionPool(host='shoplens.local', port=80): Read timed out. (read 
timeout=5)

...

⚠️ Request failed for ASIN: B0BJ47QJF6 - HTTPConnectionPool(host='shoplens.local', port=80): Read timed out. (read 
timeout=5)

⚠️ Request failed for ASIN: B0BN233BZR - HTTPConnectionPool(host='shoplens.local', port=80): Read timed out. (read 
timeout=5)

⚠️ Request failed for ASIN: B002IIE9SQ - HTTPConnectionPool(host='shoplens.local', port=80): Read timed out. (read 
timeout=5)

⚠️ Request failed for ASIN: B01M0GS8DG - HTTPConnectionPool(host='shoplens.local', port=80): Read timed out. (read 
timeout=5)

✅ Product creation process completed!


In [86]:
# Convert 'id' column to integer, handling NaNs
products_df["id"] = pd.to_numeric(products_df["id"], errors="coerce").astype("Int64")  # 'Int64' supports NaNs

products_df.to_csv("products_with_ids.csv", index=False)


In [87]:

# List rows where 'id' is null (before conversion)
missing_id_rows = products_df[products_df["id"] == 0]  # Assuming NaN was replaced with 0

# Print or save the missing rows
missing_id_rows

,id,asin,title,imgUrl,productURL,stars,reviews,price,isBestSeller,boughtInLastMonth,categoryName,categoryId
8,0,B00V636FU4,Women Ballroom Dance Shoes Mary Jane Shoes Low...,https://m.media-amazon.com/images/I/61BwR-oZLK...,https://www.amazon.co.uk/dp/B00V636FU4,4.0,78,15.00,False,0,Ballet & Dancing Footwear,20
9,0,B0CJ386B4H,2024 New -Style Shoes Social Shoe Sequins Wome...,https://m.media-amazon.com/images/I/61Bvx2Xzmd...,https://www.amazon.co.uk/dp/B0CJ386B4H,0.0,0,7.76,False,0,Ballet & Dancing Footwear,20
13,0,B00R4KTO9M,Bloch Unisex's Warm Up Boots Fashion,https://m.media-amazon.com/images/I/51ptar699B...,https://www.amazon.co.uk/dp/B00R4KTO9M,5.0,3,36.50,False,0,Ballet & Dancing Footwear,20
15,0,B0CHWFPYXZ,"Wouke Women's Cute Sneaker,Workwear Wedding Ma...",https://m.media-amazon.com/images/I/51MzyJh2Ff...,https://www.amazon.co.uk/dp/B0CHWFPYXZ,0.0,0,13.99,False,0,Ballet & Dancing Footwear,20
18,0,B0CFY8C6ZQ,AOQUNFS Girls' Dance Shoes Rhinestones Fashion...,https://m.media-amazon.com/images/I/61h8J3c+G1...,https://www.amazon.co.uk/dp/B0CFY8C6ZQ,0.0,0,29.99,False,0,Ballet & Dancing Footwear,20
...,...,...,...,...,...,...,...,...,...,...,...,...
2083,0,B07B8G7KRP,Unisex Adults’ Classic Slide Open Toe Sandals,https://m.media-amazon.com/images/I/51L2hMuQ0N...,https://www.amazon.co.uk/dp/B07B8G7KRP,4.6,8935,21.76,False,0,Women's Sports & Outdoor Shoes,40
2086,0,B0BJ47QJF6,Women's Catamount 2 Sneaker,https://m.media-amazon.com/images/I/817416RfxG...,https://www.amazon.co.uk/dp/B0BJ47QJF6,4.1,9,168.00,False,0,Women's Sports & Outdoor Shoes,40
2089,0,B0BN233BZR,"Women's GT-2000 11 Sneaker, Whisper Green Pure...",https://m.media-amazon.com/images/I/51gA1HkXNN...,https://www.amazon.co.uk/dp/B0BN233BZR,0.0,0,108.00,False,0,Women's Sports & Outdoor Shoes,40
2095,0,B002IIE9SQ,Unisex Adults’ Avenger Hiking Boot,https://m.media-amazon.com/images/I/81dH1A6a1+...,https://www.amazon.co.uk/dp/B002IIE9SQ,4.5,239,75.68,False,0,Women's Sports & Outdoor Shoes,40
